In [ ]:
## Get temperatures in x/y/z-directions from traced particles
## NOTE: When the plasma has a drifting velocity in a certain direction,
##       it is much appropriate to fit the momentum distribution in each direction,
##       and get the temperature of each direction.


# Load packages

%pylab                         
import happi                   # load data from SMILEI
from scipy.stats import norm   # fit the momentum/energy distribution


# Load data
wkdir = '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024_2/'
S = happi.Open(wkdir, reference_angular_frequency_SI=56375055300167.87)

# Prepare constants

me = 9.1e-31  # kg
mp = 1836*me  # kg
qe = 1.6e-19  # C
c  = 3.0e8    # m/s
kb = 8.6e-5   # eV/K
wr = S.namelist.w_r

In [ ]:
# Get data from the traced particles and fit them

## NOTE: The sorting of particles may take a while, 
##       and the output may be very long.
##       You may want to 'clear outputs' after this.

def fit_temperature(species, time, direction, ion):
    '''
    This function will get the momenta of the *direction*, of the *species* at the *time*.
    Parameters:
    species:   string, the one that you traced, you can find their name by `S.TrackParticles()`
    time:      float,  the simulation time in [ns]
    direction: float,  'px'/'py'/'pz'
    ion:       logic,  True/False, whether your specie is ion or not
    
    Returns:
    mom:  array, the momentum array
    wit:  array, the weight array
    mean: float, the drifting velocity in km/s
    std:  float, the temperature in eV
    '''
    
    tp = S.TrackParticles(species)                  # open the track particle diagnostic
    data = tp.getData()                             # get all the data out from the diagnostic
    tmp = data['times']                             # a tmp array for data['times']
    t_end_ns = tp.getTimes()[-1]/wr*1e9             # get the end time of the simulation and transform it into the unit of ns
    time_ns = np.linspace(0,t_end_ns,tmp.size)      # build the time array in ns.  
    timestep = int(time*1e-9*wr/100)                # get the timestep corresponding to your choice of time in ns
                                                    # NOTE: The factor `100` is really simulation-dependent.
                                                    #       One MUST make changes accordingly here.
    mom = data[direction][timestep]                 # build the array for the momentum of *direction* at all timesteps
    wit = data['w'][timestep]                       # build the array for the weight at all timesteps
    
    mom = mom[np.logical_not(np.isnan(mom))]       # remove the nan
    wit = wit[np.logical_not(np.isnan(wit))]
    
    mean, std = norm.fit(mom)             # fit the momentum at the corresponding *time*
    if ion==True:
        # transfer the output in km/s and eV for ion
        print('drift velocity = {:.1f} km/s'.format(mean*c/1e3/(mp/me)))
        print('temperature = {:.1f} eV'.format(std/kb/np.sqrt(mp/me)))
    else:
        print('drift velocity = {:.1f} km/s'.format(mean*c/1e3))
        print('temperature = {:.1f} eV'.format(std/kb/2)) 
    fit_plot(mom, wit, mean, std)                   # Plot the fitted figure to see if it works or not
    return mom, wit, mean, std

# Plot the momentum distribution and the fitted line

def fit_plot(momentum,weight,mean,std):
    '''
    Plot the fitted distribution, to check if the fitting is suitable or not
    Parameters:
    the return of the above function.
    '''
    plt.hist(momentum, bins=100, weights=weight, density=True)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    y = norm.pdf(x, mean, std)
    plt.plot(x,y)
    plt.xlabel('momentum [mec]')
    plt.ylabel('normed')
    plt.show() 

In [ ]:
# fit for all the directions and calculate the temperature

def calculate_temperature(species,time,ion):
    '''
    Calculate the average of the temperatures fitted by the momenta in all three directions
    
    Parameters:
    see the above function
    
    Returns:
    temperature: list, temperatures in each direction
    temp_ave: float, the averaged temperature
    
    Notes:
    Temperatures in each direction can be useful when you want to check if there's anisotropy,
    which may induce Weibel instability. 
    '''
    temperature = []
    for direction in ['px', 'py', 'pz']:
        mom, wit, mean, std = fit_temperature(species, time, direction, ion)
        if ion==True:
            temperature.append(std/kb/np.sqrt(mp/me))
        else:
            temperature.append(std/kb/2)
    temp_ave = sum(temperature) / 3.
    print('temperature in each direction is', temperature)
    print('temperature for '+'species'+' is', temp_ave)
    return temperature, temp_ave

In [ ]:
a, b = calculate_temperature('ion1t', 2.5, ion=True)

In [ ]:
a, b = calculate_temperature('ion2t', 2.5, ion=True)